In [1]:
import logging
from typing import Any, Set

from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.core.agent.providers import OpenAIAutomataAgent
from automata.core.singletons.dependency_factory import dependency_factory
from automata.core.tools.factory import AgentToolFactory
from automata.core.singletons.py_module_loader import py_module_loader
from automata.core.context_providers.symbol_synchronization import (
    SymbolProviderSynchronizationContext,
)
    
logger = logging.getLogger(__name__)

py_module_loader.initialize()

In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)


In [6]:
tool_dependencies

{'symbol_code_embedding_handler': <automata.core.memory_store.symbol_code_embedding.SymbolCodeEmbeddingHandler at 0x105b4d430>,
 'embedding_similarity_calculator': <automata.core.embedding.base.EmbeddingSimilarityCalculator at 0x105b4de80>,
 'symbol_doc_embedding_handler': <automata.core.memory_store.symbol_doc_embedding.SymbolDocEmbeddingHandler at 0x11fa0c760>}

In [7]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

with SymbolProviderSynchronizationContext() as synchronization_context:
    synchronization_context.register_provider(tool_dependencies["symbol_code_embedding_handler"])
    synchronization_context.synchronize()


In [8]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .build()
)

RuntimeError: Cannot get symbols before synchronization

In [ ]:
# Initialize the agent
instructions = "Explain how embeddings are used by the codebase"
agent = AutomataOpenAIAgent(instructions, config=agent_config)

In [ ]:
# Run the agent
result = agent.run()

In [ ]:
# Print the result
print(f"Result:\n{result}")